## 환경설정

In [1]:
# 필요한 라이브러리 설치
# !pip install -q openai langchain pypdf unstructured pdf2image pdfminer pypdfium2 pdfminer.six pymupdf pdfplumber amazon-textract-caller

In [2]:
# OPENAI_API
# import os

# os.environ['OPENAI_API_KEY'] = 'OPENAI API KEY 입력'

In [1]:
# 토큰 정보로드를 위한 라이브러리
# 설치: pip install python-dotenv
from dotenv import load_dotenv

# 토큰 정보로드
load_dotenv()

True

## PDF 기반 문서요약

### 문서요약 방식

PDF 의 문서를 요약하는 방식에는 문서의 크기에 따라 2가지 방식으로 나누어 볼 수 있습니다.

1. `Stuff`: 전체 문서의 내용을 모두 프롬프트의 입력으로 대입합니다. 전체 문서의 내용이 LLM 모델의 최대토큰 허용 크기보다 작은 경우, 이 방식을 선택할 수 있습니다.
2. `Map-Reduce`: 전체 문서의 내용을 쪼개서 여러의 부분 세트로 나눈 뒤, 나눈 내용을 프롬프트의 입력을 나누어 대입합니다. 예를 들어, PDF 문서가 100장으로 구성이 되어 있고, 100장 분량의 내용을 프롬프트의 입력으로 한 번에 넣을 수 없으므로, 20장씩 5개의 세트로 나눈 뒤 5번의 프롬프트 입력 후 결과로 나온 5개의 요약본을 통합하는 작업을 진행합니다.

`Stuff` 방식은 코드가 간결하지만, 현실적으로 잘 사용하게 되지 않습니다. 일반적인 PDF 문서의 전체 길이가 LLM 모델의 최대토큰 허용 크기보다 큰 경우가 대다수의 경우이기 때문입니다. 따라서, 이번 튜토리얼에서는 `Map-Reduce` 방식에 대해 자세히 다루며, `Stuff` 방식은 [여기](https://python.langchain.com/docs/use_cases/summarization#option-1-stuff) 에서 참고하실 수 있습니다.

### 흐름

> 흐름도

![](https://python.langchain.com/assets/images/summarization_use_case_3-4247e4011d0a52901288005bfaf20dc1.png)

위의 흐름도를 보면서 PDF 문서를 요약하는 흐름에 대해서 정리해 보겠습니다.

### ① 문서 로드

먼저, langchain 의 PDF 문서의 로드를 도와주는 **PDF Loader** 를 활용하여 문서를 로드합니다. langchain에서는 다양한 PDF Loader 를 제공하며, 대표적인 예시로는 `PyPDFLoader`, `PyMuPDFLoader`, `UnstructuredPDFLoader` 등이 있습니다.

여기서 문서를 로드한다는 의미는 PDF 문서의 내용을 긁어와 String 형태로 가져오는 것을 의미하며 정확하게는 `Document` 객체안에 `page_content` 속성으로 로드합니다. PDF 로드시 각종 메타데이터로 함께 로드합니다.


`PyPDFLoader` 사용하여 PDF를 로드하고, 각 문서가 페이지 내용과 페이지 번호를 포함한 메타데이터를 포함하는 문서의 배열로 만듭니다.

In [2]:
from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드
loader = PyPDFLoader("data/황순원-소나기.pdf")
document = loader.load()
document[0].page_content[:200]

'- 1 -소나기\n황순원\n소년은 개울가에서 소녀를 보자 곧 윤 초시네 증손녀 (曾孫女 )딸이라는 걸 알 수 있었다 . \n소녀는 개울에다 손을 잠그고 물장난을 하고 있는 것이다 . 서울서는 이런 개울물을 보지 \n못하기나 한 듯이.\n벌써 며칠째 소녀는 , 학교에서 돌아오는 길에 물장난이었다 . 그런데 , 어제까지 개울 기슭에\n서 하더니 , 오늘은 징검다리 한가운'

In [3]:
len(document)

7

### ② 문서 분할

문서의 내용이 긴 경우, 문서 전체의 내용을 프롬프트의 입력으로 넣을 수 없으므로 문서를 분할하는 작업을 선행합니다.

문서 분할은 토큰갯수에 따라 분할할 수 있으며, 대표적으로 많이 사용되는 모듈은 `CharacterTextSplitter`, `RecursiveCharacterTextSplitter`, `TokenTextSplitter` 등이 있습니다.

### CharacterTextSplitter

아래는 가장 단순한 방법인 `CharacterTextSplitter` 를 활용하여 문서를 분할한 예시 입니다.

이 방법은 문자(기본적으로 "\n\n")를 기반으로 분할하며, 문자의 수(chunk_size)로 길이를 측정합니다.

- 텍스트의 분할 방식: 단일 문자 기준
- chunk_size 의 측정 방식: 문자의 수로 측정

In [7]:
from langchain.text_splitter import CharacterTextSplitter

# 스플리터 지정
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n\n",  # 분할기준
    chunk_size=2000,   # 사이즈
    chunk_overlap=100, # 중첩 사이즈
)

# 분할 실행
split_docs = text_splitter.split_documents(document)

# 총 분할된 도큐먼트 수
print(f'총 분할된 도큐먼트 수: {len(split_docs)} 개')
print(f'========= 예시 =========')
print(split_docs[0].page_content)

총 분할된 도큐먼트 수: 7 개
========= 예시 =========
- 1 -소나기
황순원
소년은 개울가에서 소녀를 보자 곧 윤 초시네 증손녀 (曾孫女 )딸이라는 걸 알 수 있었다 . 
소녀는 개울에다 손을 잠그고 물장난을 하고 있는 것이다 . 서울서는 이런 개울물을 보지 
못하기나 한 듯이.
벌써 며칠째 소녀는 , 학교에서 돌아오는 길에 물장난이었다 . 그런데 , 어제까지 개울 기슭에
서 하더니 , 오늘은 징검다리 한가운데 앉아서 하고 있다.
소년은 개울둑에 앉아 버렸다 . 소녀가 비키기를 기다리자는 것이다 .
요행 지나가는 사람이 있어, 소녀가 길을 비켜 주었다 .
다음 날은 좀 늦게 개울가로 나왔다 .
이 날은 소녀가 징검다리 한가운데 앉아 세수를 하고 있었다 . 분홍 스웨터 소매를 걷어올
린 목덜미가 마냥 희었다 .
한참 세수를 하고 나더니 , 이번에는 물 속을 빤히 들여다 본다. 얼굴이라도 비추어 보는 
것이리라 . 갑자기 물을 움켜 낸다. 고기 새끼라도 지나가는 듯.
소녀는 소년이 개울둑에 앉아 있는 걸 아는지 모르는지 그냥 날쌔게 물만 움켜 낸다. 그러
나, 번번이 허탕이다 . 그대로 재미있는 양, 자꾸 물만 움킨다 . 어제처럼 개울을 건너는 사
람이 있어야 길을 비킬 모양이다 .
그러다가 소녀가 물 속에서 무엇을 하나 집어 낸다. 하얀 조약돌이었다 . 그리고는 벌떡 일
어나 팔짝팔짝 징검다리를 뛰어 건너간다 .
다 건너가더니만 홱 이리로 돌아서며 ,
“이 바보.”
조약돌이 날아왔다 .
소년은 저도 모르게 벌떡 일어섰다 .
단발 머리를 나풀거리며 소녀가 막 달린다 . 갈밭 사잇길로 들어섰다 . 뒤에는 청량한 가을 
햇살 아래 빛나는 갈꽃뿐 .
이제 저쯤 갈밭머리로 소녀가 나타나리라 . 꽤 오랜 시간이 지났다고 생각됐다 . 그런데도 
소녀는 나타나지 않는다 . 발돋움을 했다. 그러고도 상당한 시간이 지났다고 생각됐다 .
저 쪽 갈밭머리에 갈꽃이 한 옴큼 움직였다 . 소녀가 갈꽃을 안고 있었다 . 그리고 , 이제는 
천천한 걸음이었다 . 유난히 

### RecursiveCharacterTextSplitter (추천)

이 텍스트 분리기는 일반적인 텍스트에 사용하기에 **추천되는 도구** 입니다. 

이는 문자들의 리스트에 의해 매개변수화되어 있으며, 리스트에 있는 문자들을 순서대로 시도하여 청크들을 충분히 작게 만들 때까지 분리하려고 합니다. 기본 문자 리스트는 `["\n\n", "\n", " ", ""]` 입니다. 

이 분리기는 가능한 한 오래 동안 모든 단락(그리고 그 다음 문장, 그리고 그 다음 단어)을 함께 유지하려고 시도하는 효과를 가지고 있습니다. 왜냐하면 일반적으로 가장 강력하게 의미적으로 관련된 텍스트의 부분들로 보이기 때문입니다.

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 스플리터 지정
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", "(?<=\. )", " ", ""],
    chunk_size=2000,     # 사이즈
    chunk_overlap=100,   # 오버랩 사이즈
    length_function=len, # 길이 측정 함수
)

# 분할 실행
split_docs = text_splitter.split_documents(document)

# 총 분할된 도큐먼트 수
print(f'총 분할된 도큐먼트 수: {len(split_docs)} 개')
print(f'========= 예시 =========')
print(split_docs[0].page_content)

총 분할된 도큐먼트 수: 7 개
========= 예시 =========
- 1 -소나기
황순원
소년은 개울가에서 소녀를 보자 곧 윤 초시네 증손녀 (曾孫女 )딸이라는 걸 알 수 있었다 . 
소녀는 개울에다 손을 잠그고 물장난을 하고 있는 것이다 . 서울서는 이런 개울물을 보지 
못하기나 한 듯이.
벌써 며칠째 소녀는 , 학교에서 돌아오는 길에 물장난이었다 . 그런데 , 어제까지 개울 기슭에
서 하더니 , 오늘은 징검다리 한가운데 앉아서 하고 있다.
소년은 개울둑에 앉아 버렸다 . 소녀가 비키기를 기다리자는 것이다 .
요행 지나가는 사람이 있어, 소녀가 길을 비켜 주었다 .
다음 날은 좀 늦게 개울가로 나왔다 .
이 날은 소녀가 징검다리 한가운데 앉아 세수를 하고 있었다 . 분홍 스웨터 소매를 걷어올
린 목덜미가 마냥 희었다 .
한참 세수를 하고 나더니 , 이번에는 물 속을 빤히 들여다 본다. 얼굴이라도 비추어 보는 
것이리라 . 갑자기 물을 움켜 낸다. 고기 새끼라도 지나가는 듯.
소녀는 소년이 개울둑에 앉아 있는 걸 아는지 모르는지 그냥 날쌔게 물만 움켜 낸다. 그러
나, 번번이 허탕이다 . 그대로 재미있는 양, 자꾸 물만 움킨다 . 어제처럼 개울을 건너는 사
람이 있어야 길을 비킬 모양이다 .
그러다가 소녀가 물 속에서 무엇을 하나 집어 낸다. 하얀 조약돌이었다 . 그리고는 벌떡 일
어나 팔짝팔짝 징검다리를 뛰어 건너간다 .
다 건너가더니만 홱 이리로 돌아서며 ,
“이 바보.”
조약돌이 날아왔다 .
소년은 저도 모르게 벌떡 일어섰다 .
단발 머리를 나풀거리며 소녀가 막 달린다 . 갈밭 사잇길로 들어섰다 . 뒤에는 청량한 가을 
햇살 아래 빛나는 갈꽃뿐 .
이제 저쯤 갈밭머리로 소녀가 나타나리라 . 꽤 오랜 시간이 지났다고 생각됐다 . 그런데도 
소녀는 나타나지 않는다 . 발돋움을 했다. 그러고도 상당한 시간이 지났다고 생각됐다 .
저 쪽 갈밭머리에 갈꽃이 한 옴큼 움직였다 . 소녀가 갈꽃을 안고 있었다 . 그리고 , 이제는 
천천한 걸음이었다 . 유난히 

### TokenTextSplitter

`TokenTextSplitter`는 원시 텍스트 문자열을 분리하는 과정에서 먼저 텍스트를 BPE(Bite Pair Encoding) 토큰으로 변환합니다. 그리고 나서 이 토큰들을 청크로 분리하고, 하나의 청크 내에 있는 토큰들을 다시 텍스트로 변환합니다. 

이 방식을 통해, 텍스트는 더 의미 있는 단위로 분리될 수 있으며, 이는 일반적인 문자열 분리 방식보다 더 정교한 처리를 가능하게 합니다.


In [11]:
from langchain.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(chunk_size=2000,  # 사이즈
                                  chunk_overlap=100 # 오버랩 사이즈
                                  )

split_docs = text_splitter.split_documents(document)
# 총 분할된 도큐먼트 수
print(f'총 분할된 도큐먼트 수: {len(split_docs)} 개')
print(f'========= 예시 =========')
print(split_docs[0].page_content)

총 분할된 도큐먼트 수: 13 개
========= 예시 =========
- 1 -소나기
황순원
소년은 개울가에서 소녀를 보자 곧 윤 초시네 증손녀 (曾孫女 )딸이라는 걸 알 수 있었다 . 
소녀는 개울에다 손을 잠그고 물장난을 하고 있는 것이다 . 서울서는 이런 개울물을 보지 
못하기나 한 듯이.
벌써 며칠째 소녀는 , 학교에서 돌아오는 길에 물장난이었다 . 그런데 , 어제까지 개울 기슭에
서 하더니 , 오늘은 징검다리 한가운데 앉아서 하고 있다.
소년은 개울둑에 앉아 버렸다 . 소녀가 비키기를 기다리자는 것이다 .
요행 지나가는 사람이 있어, 소녀가 길을 비켜 주었다 .
다음 날은 좀 늦게 개울가로 나왔다 .
이 날은 소녀가 징검다리 한가운데 앉아 세수를 하고 있었다 . 분홍 스웨터 소매를 걷어올
린 목덜미가 마냥 희었다 .
한참 세수를 하고 나더니 , 이번에는 물 속을 빤히 들여다 본다. 얼굴이라도 비추어 보는 
것이리라 . 갑자기 물을 움켜 낸다. 고기 새끼라도 지나가는 듯.
소녀는 소년이 개울둑에 앉아 있는 걸 아는지 모르는지 그냥 날쌔게 물만 움켜 낸다. 그러
나, 번번이 허탕이다 . 그대로 재미있는 양, 자꾸 물만 움킨다 . 어제처럼 개울을 건너는 사
람이 있어야 길을 비킬 모양이다 .
그러다가 소녀가 물 속에서 무엇을 하나 집어 낸다. 하얀 조약돌이었다 . 그리고는 벌떡 일
어나 팔짝팔짝 징검다리를 뛰어 건너간다 .
다 건너가더니만 홱 이리로 돌아서며 ,
“이 바보.”
조약돌이 날아왔다 .
소년은 저도 모르게 벌떡 일어섰다 .
단발 머리를 나풀거리며 소녀가 막 달린다 . 갈밭 사잇길로 들어섰다 . 뒤에는 청량한 가을 
햇살 아래 빛나는 갈꽃뿐 .
이제 저쯤 갈밭머리로 소녀가 나타나리라 . 꽤 오랜 시간이 지났다고 생각됐다 . 그런데도 
소녀는 나타나지 않는다 . 발돋움을 했다. 그러고도 상당한 시간이 지났다고 생각됐다 .
저 쪽 갈밭머리에 갈꽃이 한 옴큼 움직였다 . 소녀가 갈꽃을 안고 있었다 . 그리고 , 이제는 
천천한 걸음이었다 . 유난히

### ③ 분할된 각 문서에 대한 요약 실행

분할된 문서에 요약(summarization) 과 같은 테스크를 수행하는 것을 **Map**, 각 문서의 요약본을 하나로 통합하는 작업을 **Reduce** 라고 부릅니다. 즉, **Map-Reduce** 방식은 문서를 **분할 - 요약 - 통합** 을 수행하게 됩니다.

그 중 Map 단계에서 수행할 프롬프트를 템플릿으로 정의하고 chain 을 생성해 보겠습니다.

In [14]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

# Map 단계에서 처리할 프롬프트 정의
# 분할된 문서에 적용할 프롬프트 내용을 기입합니다.
# 여기서 {pages} 변수에는 분할된 문서가 차례대로 대입되니다.
map_template = """다음은 문서 중 일부 내용입니다
{pages}
이 문서 목록을 기반으로 주요 내용을 요약해 주세요.
답변:"""

# Map 프롬프트 완성
map_prompt = PromptTemplate.from_template(map_template)

# Map에서 수행할 LLMChain 정의
llm = ChatOpenAI(temperature=0, 
                 model_name='gpt-3.5-turbo-16k')
map_chain = LLMChain(llm=llm, prompt=map_prompt)

### ④ 각 문서의 요약본에 대한 통합

Reduce 단계는 이전 단계인 Map 단계에서 분할된 문서에 대한 요약본을 **통합처리** 하는 역할을 수행합니다.

Reduce 단계에서는 통합처리시 필요한 프롬프트를 정의합니다. 통합처리시 단순하게 전체 요약본 테스크를 수행할 수 도 있고, 요약본을 토대로 보고서, 이메일, 뉴스레터, 독서감상문 등의 다양한 테스크를 수행하도록 할 수 있습니다.

아래는 통합 요약본을 작성하는 프롬프트 예시입니다.

In [15]:
# Reduce 단계에서 처리할 프롬프트 정의
reduce_template = """다음은 요약의 집합입니다:
{doc_summaries}
이것들을 바탕으로 통합된 요약을 만들어 주세요.
답변:"""

# Reduce 프롬프트 완성
reduce_prompt = PromptTemplate.from_template(reduce_template)

# Reduce에서 수행할 LLMChain 정의
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

`combine_documents_chain` 은 Reduce 작업을 수행하는 체인입니다.

Map 단계에서 완성된 분할된 요약본이 `StuffDocumentsChain` 의 `doc_summaries` 입력으로 주입됩니다.

`ReduceDocumentsChain` 은 주입된 입력 값을 통합하는 작업을 수행합니다.

In [17]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains import ReduceDocumentsChain

# 문서의 목록을 받아들여, 이를 단일 문자열로 결합하고, 이를 LLMChain에 전달합니다.
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain,                
    document_variable_name="doc_summaries" # Reduce 프롬프트에 대입되는 변수
)

# Map 문서를 통합하고 순차적으로 Reduce합니다.
reduce_documents_chain = ReduceDocumentsChain(
    # 호출되는 최종 체인입니다.
    combine_documents_chain=combine_documents_chain,
    # 문서가 `StuffDocumentsChain`의 컨텍스트를 초과하는 경우
    collapse_documents_chain=combine_documents_chain,
    # 문서를 그룹화할 때의 토큰 최대 개수입니다.
    token_max=4000,
)

### 통합체인(Combined Chain) 생성

통합체인 생성 단계에서는 이전에 정의한 Map 체인과 Reduce 체인을 연결하고, `MapReduceDocumentsChain` 객체를 통해 통합하는 단계입니다.

In [18]:

from langchain.chains import MapReduceDocumentsChain

# 문서들에 체인을 매핑하여 결합하고, 그 다음 결과들을 결합합니다.
map_reduce_chain = MapReduceDocumentsChain(
    # Map 체인
    llm_chain=map_chain,
    # Reduce 체인
    reduce_documents_chain=reduce_documents_chain,
    # 문서를 넣을 llm_chain의 변수 이름(map_template 에 정의된 변수명)
    document_variable_name="pages",
    # 출력에서 매핑 단계의 결과를 반환합니다.
    return_intermediate_steps=False,
)

In [19]:
# Map-Reduce 체인 실행
# 입력: 분할된 도큐먼트(②의 결과물)
result = map_reduce_chain.run(split_docs)
# 요약결과 출력
print(result)

소년은 개울가에서 소녀를 보고 그녀가 윤 초시네 증손녀라는 것을 알게 됩니다. 소녀는 개울에서 물장난을 하고 있으며, 소년은 그녀를 지켜보고 있습니다. 소녀는 물 속을 들여다보고 물을 움켜 낸 후 조약돌을 집어내고 징검다리를 건너갑니다. 소녀는 갈꽃을 안고 갈꽃머리로 걸어가고, 소년은 소녀의 그림자가 뵈지 않는 날이 계속되면서 허전함을 느끼고 주머니 속 조약돌을 가지고 다닙니다. 소년은 물장난을 하던 징검다리에 앉아 물 속을 들여다보고 세수를 하지만 자신의 얼굴이 검게 비치는 것을 싫어합니다. 소년은 물 속에서 얼굴을 움키다가 소녀를 발견하고 달리기를 시작합니다. 소년은 메밀밭을 지나가며 코피를 훔치고, 소녀와 함께 산으로 가기로 결정합니다. 소녀와 소년은 허수아비를 만나고 놀며 산으로 달려갑니다. 소년은 독수리가 맴돌아 어지러워합니다. 소녀와 소년이 함께 농장을 돌아다니며 여러 꽃과 작물을 보고 먹습니다. 소녀는 참외를 맛보고 맵다고 하며 버립니다. 소년은 소녀에게 꽃을 건네주고 함께 산으로 가서 더 많은 꽃을 꺾습니다. 소녀는 마타리꽃을 좋아하며 소년에게 꽃을 버리지 말라고 합니다. 소녀와 소년은 산마루에서 휴식을 취하며 주위의 조용함과 가을 햇살을 즐깁니다. 소녀는 칡덩굴의 꽃을 보고 등나무를 떠올리며 감동합니다. 소년과 소녀가 함께 놀다가 비가 오기 시작합니다. 소년은 소녀를 비에서 보호하기 위해 원두막으로 이동합니다. 소년은 비를 맞으면서 소녀를 보호하고, 소녀는 소년의 도움을 받아 편안하게 있습니다. 소년은 수숫단을 이용하여 소녀를 더욱 안전하게 보호합니다. 소년과 소녀가 비에 젖은 상황에서 만나며 소녀가 소년을 안아주고 소년은 소녀에게 마음이 끌립니다. 소녀는 개울가에서 소년을 기다리지 않고, 소년은 소녀를 찾아다니며 마음을 품고 있습니다. 소녀는 소나기 때문에 아프다고 하고, 소년은 소녀를 위해 대추를 받습니다. 소녀는 제사를 지낸 후 집을 내주게 됩니다. 소년은 소녀네가 이사해 오기 전에 벌써 어른들의 이야기를 들어서, 윤 초시 손자가 서울에서

## 전체코드

In [20]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains import ReduceDocumentsChain, MapReduceDocumentsChain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# ========== ① 문서로드 ========== #

# PDF 파일 로드
loader = PyPDFLoader("data/황순원-소나기.pdf")
document = loader.load()

# ========== ② 문서분할 ========== #

# 스플리터 지정
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", "(?<=\. )", " ", ""],
    chunk_size=2000,
    chunk_overlap=100,
    length_function=len, 
)


# 분할 실행
split_docs = text_splitter.split_documents(document)
# 총 분할된 도큐먼트 수
print(f'총 분할된 도큐먼트 수: {len(split_docs)}')

# ========== ③ Map 단계 ========== #

# Map 단계에서 처리할 프롬프트 정의
# 분할된 문서에 적용할 프롬프트 내용을 기입합니다.
# 여기서 {pages} 변수에는 분할된 문서가 차례대로 대입되니다.
map_template = """다음은 문서 중 일부 내용입니다
{pages}
이 문서 목록을 기반으로 주요 내용을 요약해 주세요.
답변:"""

# Map 프롬프트 완성
map_prompt = PromptTemplate.from_template(map_template)

# Map에서 수행할 LLMChain 정의
llm = ChatOpenAI(temperature=0, 
                 model_name='gpt-3.5-turbo-16k')
map_chain = LLMChain(llm=llm, prompt=map_prompt)

# ========== ④ Reduce 단계 ========== #

# Reduce 단계에서 처리할 프롬프트 정의
reduce_template = """다음은 요약의 집합입니다:
{doc_summaries}
이것들을 바탕으로 통합된 요약을 만들어 주세요.
답변:"""

# Reduce 프롬프트 완성
reduce_prompt = PromptTemplate.from_template(reduce_template)

# Reduce에서 수행할 LLMChain 정의
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# 문서의 목록을 받아들여, 이를 단일 문자열로 결합하고, 이를 LLMChain에 전달합니다.
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain,                
    document_variable_name="doc_summaries" # Reduce 프롬프트에 대입되는 변수
)

# Map 문서를 통합하고 순차적으로 Reduce합니다.
reduce_documents_chain = ReduceDocumentsChain(
    # 호출되는 최종 체인입니다.
    combine_documents_chain=combine_documents_chain,
    # 문서가 `StuffDocumentsChain`의 컨텍스트를 초과하는 경우
    collapse_documents_chain=combine_documents_chain,
    # 문서를 그룹화할 때의 토큰 최대 개수입니다.
    token_max=4000,
)

# ========== ⑤ Map-Reduce 통합단계 ========== #

# 문서들에 체인을 매핑하여 결합하고, 그 다음 결과들을 결합합니다.
map_reduce_chain = MapReduceDocumentsChain(
    # Map 체인
    llm_chain=map_chain,
    # Reduce 체인
    reduce_documents_chain=reduce_documents_chain,
    # 문서를 넣을 llm_chain의 변수 이름(map_template 에 정의된 변수명)
    document_variable_name="pages",
    # 출력에서 매핑 단계의 결과를 반환합니다.
    return_intermediate_steps=False,
)

# ========== ⑥ 실행 결과 ========== #

# Map-Reduce 체인 실행
# 입력: 분할된 도큐먼트(②의 결과물)
result = map_reduce_chain.run(split_docs)
# 요약결과 출력
print(result)

총 분할된 도큐먼트 수: 7
소년은 개울가에서 소녀를 만나게 되고, 그녀가 윤 초시네 증손녀라는 것을 알게 된다. 소녀는 물장난을 하고 있는데, 소년은 그녀를 지켜보고 있다. 소녀는 물 속을 들여다보고 물을 움켜 낸다. 소년은 소녀가 물 속에서 조약돌을 집어내고 징검다리를 건너간다는 것을 보고 따라간다. 소녀는 갈꽃을 안고 갈꽃머리로 걸어간다. 소년은 소녀의 그림자가 뵈지 않는 날이 계속되면서 허전함을 느끼고 주머니 속 조약돌을 가지고 다닌다. 소년은 소녀가 물장난을 하던 징검다리에 앉아 물 속을 들여다보고 세수를 한다, 그러나 자신의 얼굴이 검게 비치는 것을 싫어한다. 소년은 물 속에서 얼굴을 움키다가 소녀를 발견하고 달리기를 시작한다. 소년은 메밀밭을 지나가며 코피를 훔치고, 소녀와 함께 산으로 가기로 결정한다. 소녀와 소년은 허수아비를 만나고 놀며 산으로 달려간다. 소년은 독수리가 맴돌아 어지러워한다. 소녀와 소년이 함께 농장을 돌아다니며 여러 꽃과 작물을 보고 먹는다. 소녀는 참외를 맛보고 맵다고 하며 버린다. 소년은 소녀에게 꽃을 건네주고 함께 산으로 가서 더 많은 꽃을 꺾는다. 소녀는 마타리꽃을 좋아하며 소년에게 꽃을 버리지 말라고 한다. 소녀와 소년은 산마루에서 휴식을 취하며 주위의 조용함과 가을 햇살을 즐긴다. 소녀는 칡덩굴의 꽃을 보고 등나무를 떠올리며 감동한다. 소년과 소녀가 함께 놀다가 비가 오기 시작한다. 소년은 소녀를 비에서 보호하기 위해 원두막으로 이동한다. 소년은 비를 맞으면서 소녀를 보호하고, 소녀는 소년의 도움을 받아 편안하게 있다. 소년은 수숫단을 이용하여 소녀를 더욱 안전하게 보호한다. 소년과 소녀가 비에 젖은 상황에서 만나게 되고, 소녀가 소년을 안아주고 소년은 소녀에게 마음이 끌린다. 그러나 소녀는 개울가에서 소년을 기다리지 않고 집을 나가게 된다. 소년은 소녀네가 이사해 오기 전에 벌써 어른들의 이야기를 들어서, 윤 초시 손자가 서울에서 사업에 실패해 고향에 돌아오지 않을 수 없게 되었다는 걸 알고 있었다. 소년은 이사 

이전 단계의 결과물인 문서의 통합요약본을 바탕으로 다음과 같은 추가 Task를 수행할 수 있습니다. 아래는 독서감상문 작성의 예시입니다.

In [21]:
# 질문 템플릿 형식 정의
template = """다음은 소설에 대한 요약본입니다. 
다음의 내용을 독서 감상문 형식으로 작성해 주세요. 

독서 감상문의 형식은 다음과 같습니다:

처음: 글을 읽게 된 동기나 책을 처음 대했을 때의 느낌을 쓰고, 글의 종류나 지은이 소개, 주인
공이나 주제의 소개
중간: 주인공의 행동과 나의 행동을 비교해 보기도 하고, 글의 내용을 평가해 보기도 하며, 글
속에서 발견한 주제나 의미가 우리 사회에 어떻게 작용할 것인가를 씁니다. 그리고 글을 읽으면서 받은
감동을 쓰기도 합니다.
끝: 글의 내용을 정리하며, 교훈을 적어두기도 한다. 그리고 끝글은 지루하지 않도록 산뜻하게

{text}

답변:
"""

# 템플릿 완성
prompt = PromptTemplate(template=template, input_variables=['text'])

# 연결된 체인(Chain)객체 생성
llm_chain = LLMChain(prompt=prompt, llm=llm)

output = llm_chain.run(text=result)
print(output)

처음에 이 소설을 읽게 된 동기는 소설의 제목과 내용이 궁금해서였습니다. 소년과 소녀의 이야기가 어떻게 전개될지, 그리고 그들의 만남이 어떤 의미를 지니는지 알고 싶었습니다. 이 소설은 김동인 작가의 작품으로, 소년과 소녀의 만남을 통해 사랑과 희망의 의미를 탐구하는 내용을 담고 있습니다.

이 소설을 읽으면서 주인공과 나의 행동을 비교해 보았습니다. 주인공은 소녀를 보고 그녀를 따라가며 그녀를 지켜보고 있었는데, 나는 주변 사람들을 그렇게 관찰하고 지켜보는 습관이 없었습니다. 주인공은 소녀의 행동을 따라가며 그녀와 함께 여러 경험을 하고 있었는데, 나는 주변 사람들과의 교류를 더 많이 추구해야겠다는 생각이 들었습니다.

이 소설은 소년과 소녀의 만남을 통해 사랑과 희망의 의미를 탐구합니다. 소년은 소녀와 함께 여러 경험을 하면서 행복함을 느끼고, 소녀의 존재가 자신에게 희망을 주는 것을 느낍니다. 이러한 주제와 의미는 우리 사회에도 적용될 수 있습니다. 사람들은 서로의 존재와 관계를 통해 행복과 희망을 얻을 수 있으며, 이를 통해 사회 전체가 더욱 발전할 수 있다는 것을 알 수 있습니다.

이 소설을 읽으면서 받은 감동은 소년과 소녀의 순수하고 진심된 사랑에 있었습니다. 소년은 소녀를 보호하고 함께 힘들고 어려운 상황을 극복하려고 노력합니다. 이러한 사랑과 헌신은 독자에게 큰 감동을 주었고, 나 역시 이러한 사랑을 배울 수 있었습니다.

이 소설의 내용을 정리하면, 소년과 소녀의 만남을 통해 사랑과 희망의 의미를 탐구하는 이야기입니다. 소년은 소녀와 함께 여러 경험을 하면서 행복함과 희망을 느끼고, 이를 통해 우리 사회에도 사랑과 희망을 전파할 수 있다는 교훈을 얻을 수 있습니다. 이 소설은 산뜻하고 감동적인 이야기로, 독자에게 큰 감동을 주는 작품입니다.
